# Environment configuration

In [23]:
# Upgrading pip which will be used to install all libraries
!pip install -r requirements.txt

  Using cached pymongo-4.3.3-cp39-cp39-win_amd64.whl (382 kB)
  Using cached transformers-4.25.1-py3-none-any.whl (5.8 MB)
  Using cached huggingface_hub-0.11.1-py3-none-any.whl (182 kB)
  Using cached filelock-3.9.0-py3-none-any.whl (9.7 kB)
  Attempting uninstall: pymongo
    Found existing installation: pymongo 3.12.3
    Uninstalling pymongo-3.12.3:
      Successfully uninstalled pymongo-3.12.3


In [1]:
# to check if working on GPU
# !pip install DeepSaki
from tensorflow import keras
import DeepSaki
strategy, RUNTIME_ENVIRONMENT, hw_accelerator_handle = DeepSaki.utils.DetectHw()

Running on single GPU  /device:GPU:0
Number of accelerators:  1
____________________________________________________________________________________
Device List: 
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7780481143290808895
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1721342363
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3192076693052380548
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3050 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


## Import songs with lyrics from MongoDB

In [5]:
import pandas as pd

In [2]:
from typing import List
# from ../models/msd_song import MsdSongWithLyrics
from models.msd_song import MsdSongWithLyrics
from dao.dao_msd_songs_with_lyrics import DAOMsdSongsWithLyrics

dao_songs_with_lyrics: DAOMsdSongsWithLyrics = DAOMsdSongsWithLyrics(name="songs_with_lyrics")
songs: List[MsdSongWithLyrics] = dao_songs_with_lyrics.find_all()

In [6]:
headers = songs[0].dict().keys()
songs_values = [song.dict().values() for song in songs]
songs_df = pd.DataFrame(songs_values, columns=headers)
songs_df.head()

,song_id,title,release,artist_name,year,tag,features,lyrics
0,SOLJTLX12AB01890ED,El hijo del pueblo,32 Grandes Éxitos CD 2,Jorge Negrete,1997,pop,[],Es mi orgullo haber nacido en el barrio más hu...
1,SOMPVQB12A8C1379BB,Pilots,The Loyal,Tiger Lou,2005,pop,[],Raise the chandelier light the candels dear i ...
2,SOSDCFG12AB0184647,006,Lena 20 År,Lena Philipsson,1998,pop,[],I had come in the name of love\nWith a mission...
3,SOKOVRQ12A8C142811,Ethos of Coercion,Descend Into Depravity,Dying Fetus,2009,rock,[],"Castigation of the offenders, no punishment ou..."
4,SOIMMJJ12AF72AD643,Rock-N-Rule,I'm Only A Man (Bonus Track Version),Emery,2007,rock,[],[Intro]\nThis is a waking up\nThis is your fin...


In [19]:
from dao.dao_msd_triplets import DAOMsdTriplets
from models.msd_triplet import MsdTriplet
dao_songs: DAOMsdTriplets = DAOMsdTriplets()

done = 0
count = 0
for song_id in songs_df['song_id']:
    triplets: List[MsdTriplet] = dao_songs.find_many_by_query({'song_id': song_id})
    if len(triplets) != 0:
        count += 1
    done += 1
    if not done % 10:
        print(done)
print(count) # Everything is empty :(( -> cant use supervised sentiment

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380


KeyboardInterrupt: 

In [4]:
from tensorflow.python.client.session import InteractiveSession
from tensorflow.core.protobuf.config_pb2 import GPUOptions, ConfigProto
from transformers import pipeline

gpu_options = GPUOptions(allow_growth=True)
session = InteractiveSession(config=ConfigProto(gpu_options=gpu_options))
sentiment_analysis = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")
print(sentiment_analysis("I love my girl"))

ResourceExhaustedError: Exception encountered when calling layer "roberta" "                 f"(type TFRobertaMainLayer).

{{function_node __wrapped__StatelessTruncatedNormalV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[50265,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:StatelessTruncatedNormalV2]

Call arguments received by layer "roberta" "                 f"(type TFRobertaMainLayer):
  • input_ids=tf.Tensor(shape=(3, 5), dtype=int32)
  • attention_mask=None
  • token_type_ids=None
  • position_ids=None
  • head_mask=None
  • inputs_embeds=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • past_key_values=None
  • use_cache=None
  • output_attentions=False
  • output_hidden_states=False
  • return_dict=True
  • training=False